In [2]:
import requests
import pandas as pd 
import numpy as np
import json
import time

api_key = "6df661d9964de19e4ca7721d9cce69a6"
url_api = "https://api.themoviedb.org/3"

In [3]:
def get(string):
    return requests.get(url_api + string + "?api_key=" + api_key)

def get_movie_id(id):
    return get(f"/movie/{id}")

def get_casting_id(id):
    return get(f"/movie/{id}/credits")

def search_movie(name):
    return requests.get(url_api + "/search/movie" + "?api_key=" + api_key + "&query=" + name)


def get_collections(id):
    return get(f"/collection/{id}")

#print(get_collections(531241).text)
#print(get_movie_id(634649).text)
#print(search_movie("Spider-Man"))



In [5]:
# Construction de la database

columns = ['date', 'titre', 'budget', 'recette', 'duree', 'genres', 'synopsis', 'tagline', 'realisateur', 'casting', 'compagnies_production', 'suite', 'production_countries', 'langues', 'imdb_id', 'vote_average', 'vote_count', 'status' ]

def build_base(start, end):
    array = []
    
    for id in range(start, end):
        response = get_movie_id(id)
        
        if response.status_code == requests.codes.OK :
            dict = json.loads(response.text)
            features = []
            if dict["budget"] != 0 or dict["revenue"] != 0:
                #On ajoute les features
                features.append(dict["release_date"])
                features.append(dict["original_title"])
                features.append(dict["budget"])
                features.append(dict["revenue"])
                

                features.append(dict["runtime"])

                genres = []
                for ele in dict["genres"]:
                    genres.append(ele['name'])

                features.append(genres)
                features.append(dict['overview'])
                features.append(dict["tagline"])


                #Casting, réalisateur
                response = get_casting_id(id)
                cast = []
                realisateur = []
                if response.status_code == requests.codes.OK :
                    dict_casting = json.loads(response.text)
                    for ele in dict_casting["cast"]:
                        if ele["known_for_department"] == "Acting":
                            cast.append(ele["name"])
                    
                    for ele in dict_casting["crew"]:
                        if ele["known_for_department"] == "Directing":
                            if "job" in ele.keys():
                                if ele["job"] == "Director":
                                    realisateur.append(ele["name"])
                features.append(realisateur)
                features.append(cast)


                #Compagnies_production
                compagnies_production = []
                for ele in dict["production_companies"]:
                    compagnies_production.append(ele["name"])
                features.append(compagnies_production)


                #Suite
                if dict["belongs_to_collection"] != None:
                    response = get_collections(dict["belongs_to_collection"]["id"])
                    if response.status_code == requests.codes.OK :
                        dict_collection = json.loads(response.text)
                        k = 1
                        for ele in dict_collection["parts"]:
                            if ele["original_title"] == dict["original_title"]:
                                features.append(k)
                                break
                            k += 1
                else : 
                    features.append(1)
                #Production_countries
                production_countries = []
                for ele in dict['production_countries']:
                    production_countries.append(ele["iso_3166_1"])
                
                features.append(production_countries)


                #Langues
                langues = []
                for ele in dict["spoken_languages"]:
                    langues.append(ele["iso_639_1"])
                
                features.append(langues)

                #imdb_id

                features.append(dict["imdb_id"])

                #vote_average
                features.append(dict["vote_average"])

                #vote_count
                features.append(dict["vote_count"])

                #status
                features.append(dict["status"])



                #Ajout ligne
                array.append(features)



    df = pd.DataFrame(np.array(array),columns=columns)
    df.to_csv(f"csv/{start}_{end}.csv")

    return 
build_base(1,100)

C:\Users\kerac\AppData\Local\Temp\ipykernel_13696\755748901.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df = pd.DataFrame(np.array(array),columns=columns)


In [22]:
#Construction

def build_all_base(debut):
    for k in range(debut,1000000, 1000):
        if k==1:
            build_base(1, 1000)
        else: 
            build_base(k, k+1000)
        time.sleep(600)

In [23]:
build_all_base(1)

C:\Users\kerac\AppData\Local\Temp\ipykernel_4124\472915361.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df = pd.DataFrame(np.array(array),columns=columns)
